In [ ]:
import pandas as pd
import re
import requests
import urllib
import nltk
from nltk.corpus import stopwords
import pymorphy2
import warnings
warnings.filterwarnings("ignore")

#nltk.download('stopwords')
#nltk.download('punkt')

In [ ]:
separator = r'\', \''
frame_columns_names = ["Тема обращения","Суть обращения",
                        "Чиновник","Должность",
                        "Дата подачи заявления"]
df = pd.read_csv('Mosru_dump_all_pages.csv',
                    encoding = 'utf-8',
                    sep = separator,
                    names = None,
                    skiprows = 1)
df.columns = frame_columns_names

In [ ]:
stop_words = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def clean_string(string):
    return re.sub('\'','',string)

In [ ]:
def search_theme(string):
    res = re.findall('«.*»',string)
    return res[0] if len(res)>0 else ""

In [ ]:
def search_address(string):
    res = re.findall('по адресу\s?(.*$)',string)
    return res[0] if len(res)>0 else ""

In [ ]:
def preprocessingText(line):
    line = line.lower()
    line = re.sub(r'\d+','',line,flags=re.UNICODE)#delete numbers
    line = re.sub(r'[^\w\s]',' ',line,flags=re.UNICODE)
    tl = nltk.word_tokenize(line) #tl - tokenize line
    nft = [morph.parse(token)[0].normal_form for token in tl if len(token)>1]#nft - normal form token
    clean_words_list = [normalToken for normalToken in nft if normalToken not in stop_words and len(normalToken)>1]#преобразованный список
    return ' '.join(clean_words_list)

In [ ]:
for column in frame_columns_names:
    df[column] = df[column].apply(clean_string)

In [ ]:
df["theme"] = df['Тема обращения'].apply(search_theme)

In [ ]:
df["address"] = df['Тема обращения'].apply(search_address)

In [ ]:
df['Obr_clean'] = df['Суть обращения'].apply(preprocessingText)

In [ ]:
df.head(2)

In [ ]:
df.to_csv('clean_frame.csv',sep='|',encoding='utf-8')

Вторая часть после загрузки и обработки данных

In [ ]:
n_df = pd.read_csv('clean_frame.csv', sep='|', encoding='utf-8')

In [ ]:
unique_addresses = n_df['address'].unique().tolist()

In [ ]:
mapbox_token = "your_mapbox_token"

In [ ]:
def search_district(address):
    url = "https://api.mapbox.com/geocoding/v5/mapbox.places/{0}.json?access_token={1}".format(urllib.parse.quote("Москва "+address), mapbox_token)
    response = requests.get(url)
    district = ""
    try:
        district = response.json()['features'][0]['place_name']
    except Exception as error:
        print("Возникла ошибка"+str(error))
    finally:
        return district

In [ ]:
for address in unique_addresses[123:129]:
    print(search_district(address))